In [1]:
from collections import namedtuple

import itertools as itt
from tqdm import tnrange,tqdm_notebook

from ipywidgets import interact

from funcs import gps_coord, gln_coord, get_epoch_array, file_byte_iterator,  filter_epoch, general_parser, map_rec, Uchet

%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
from bokeh.palettes import Category10
from scipy import signal
import scipy.io as io
from bokeh.transform import jitter
TOOLS = "pan,wheel_zoom,box_zoom,reset,box_select,lasso_select, save"
output_notebook()
#output_file('html/b1_{}.html'.format(filename[5:7]))

Loading BokehJS ...

In [3]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import FancyArrowPatch
import numpy as np
from mpl_toolkits.mplot3d import proj3d
from itertools import product, combinations

class Arrow3D(FancyArrowPatch):

    def __init__(self, xs, ys, zs, *args, **kwargs):
        FancyArrowPatch.__init__(self, (0, 0), (0, 0), *args, **kwargs)
        self._verts3d = xs, ys, zs

    def draw(self, renderer):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))
        FancyArrowPatch.draw(self, renderer)

In [4]:
def get(filename, sv_arr):
    data = bytes(file_byte_iterator(filename))
    epoch_array = get_epoch_array(data)
    req_ids = [b'~~', b'SI', b'AN', b'rc', b'pc', b'RC', b'PC', b'NE', b'PV', b'NN', b'NT', b'EL', b'AZ']
    epoch_min_arr = [filter_epoch(req_ids, epoch) for epoch in epoch_array]

    pseudo_params = namedtuple('pseudo_params', ['usi','cp', 'pr', 'elev'])
    ephemerisNE = namedtuple('ephemerisNE', ['sv', 'frqNum', 'dne',
                             'tk', 'tb', 'health', 'age', 'flags',
                             'rx', 'ry', 'rz', 'vx', 'vy', 'vz', 'wx', 'wy', 'wz',
                             'tauSys', 'tau', 'gamma', 'cs'])

    def pick_elem(dict):
        return [dict[b'SI'].usi, dict[b'PC'].cp, dict[b'RC'].pr, dict[b'EL'].elev]

    def process_epoch(epoch):
        ep = [map_rec(general_parser(rec)) for rec in epoch]
        ep_dict = {rec.id: rec for rec in ep}
        processed_data = {
            'RT': (ep_dict[b'~~'].tod[0]*1e-3)%86400,
            'NT': (ep_dict[b'NT'].tod*1e-3)%86400,
            'pseudo_params': [pseudo_params(usi, cp, pr, elev) for usi, cp, pr, elev in zip( *pick_elem(ep_dict) )],
            'receiver': ep_dict[b'PV'],
            'NN': ep_dict[b'NN'],
        }
        if b'NE' in ep_dict:
            processed_data['ephemerisNE'] = ep_dict[ b'NE']
        return processed_data

    sats_array = [process_epoch(epoch) for epoch in epoch_min_arr]
    #sat_live = [[find_sat(sv, sats['pseudo_params']) for sv in sv_arr] for sats in sats_array]    
    
    return sats_array

fR = lambda xr,yr,zr: np.sqrt((xr)**2+(yr)**2+(zr)**2)

def sync(data, data1):
    epoch = 0
    X,Y = 0,0
    if data[0]['RT'] > data1[0]['RT']:
        while data[0]['RT'] != data1[epoch]['RT']:
            epoch += 1
        X,Y = data, data1[epoch:]
    else:
        while data1[0]['RT'] != data[epoch]['RT']:
            epoch += 1
        X,Y =  data[epoch:], data1
    if len(X) > len(Y):
        X = X[:len(Y)]
    else:
        Y = Y[:len(X)]
    return X,Y

In [5]:
pseudo_params = namedtuple('pseudo_params', ['usi','cp', 'pr', 'elev'])
def find_sat(usi, sats):
    return next((sat for sat in sats if (sat.usi == usi)), pseudo_params(usi, 0, 0, 0))
def popr(filename):
    if '1' in filename:
        return 0.169, 0.829, -0.452  
    elif '2' in filename:
        return -0.642, 1.169, -0.139  
    elif '3' in filename:
        return -0.839, 0.351, 0.293  

def select_eph(target, arr):
    compare = lambda x: abs(target - x)
    return sorted(arr, key=compare)[0]

In [6]:
def StatusGLN(sys, sys0,  x0 = 0, y0 = 0, z0 = 0):
    Status = []
    for epoch in tnrange(len(sys0)):
        stat = []
        #rt = int((sys[epoch]['RT'].tod[0]*1e-3)%86400)
        rt = sys[epoch]['RT']
        nt = sys[epoch]['NT']
        pv = B1[0]['receiver']
        pv0 = sys[0]['receiver']
        sats = sys0[epoch]#['pseudo_params']
        for usi in sv_gln:
            if sats[usi].usi in EphemerisNE.keys():
                cur_eph = EphemerisNE[sats[usi].usi][select_eph(nt, EphemerisNE[sats[usi].usi].keys())]
                sv = [*EphemerisNE[sats[usi].usi].values()][0].sv
                X,Y,Z = Uchet(*gln_coord(nt, 5, cur_eph, sats[usi].pr), pv.x+x0,pv.y+y0,pv.z+z0)
                R = fR( X-(pv.x+x0), Y-(pv.y+y0), Z-(pv.z+z0) )
                stat.append(status(rt, sats[usi].usi, sv, X, Y, Z, pv0.x, pv0.y, pv0.z, R, sats[usi].cp, sats[usi].pr, sats[usi].elev))
            else:
                stat.append(status(rt, sats[usi].usi, 0, 0,0,0,0,0,0, 0,(sats[usi].cp),sats[usi].pr, 0))
        Status.append({rec.usi: rec for rec in stat if rec.X != 0})
    return Status

In [7]:
status = namedtuple('status',['rt','usi', 'sv' ,'X', 'Y', 'Z', 'x', 'y', 'z' ,'R','cp', 'pr', 'el'])
status0 = namedtuple('status',['rt', 'usi', 'cp', 'R'])

d_x0 = namedtuple('d_x',['d_fi','d_R', 'd_p'])
dd_x0 = namedtuple('dd_x',['dd_fi','dd_R', 'dd_p'])

In [8]:
sv_gps = [*range(1,39)]
sv_gln = [*range(38,70)]
sv_gln_ = [*range(-7,24)]
sv_arr0 = [47, 50, 55, 57]
sv_arr = [4, 5, 6, 24, 25, 30]
F = 1575.42e6
C = 299792458.
lam = C / F
lam_gln = {usi: C / ((1602 + (usi - 45) * 0.5625)*1e6) for usi in sv_gln}
F_gln = {usi: ((1602 + (usi - 45) * 0.5625)*1e6) for usi in sv_gln}

In [9]:
%%time
filename = 'data/r1.dat'
B1,R1 = sync(get('data/b1.dat', sv_gps), get(filename, sv_gps))

epne = [sat['ephemerisNE'] for sat in B1 if 'ephemerisNE' in sat]

EphemerisNE = {rec.frqNum+45: {ep.tb: ep for ep in epne if ep.frqNum == rec.frqNum} for rec in epne}

Wall time: 3.31 s


In [10]:
dx,dy,dz = popr(filename)

bgln = {epoch: {usi: find_sat(usi, B1[epoch]['pseudo_params']) for usi in sv_gln} for epoch in range(len(B1))}
rgln = {epoch: {usi: find_sat(usi, R1[epoch]['pseudo_params']) for usi in sv_gln} for epoch in range(len(B1))}

In [11]:
%%time
Bgln = StatusGLN(B1, bgln)
Rgln = StatusGLN(R1, rgln, dx, dy, dz)


Wall time: 44 s


In [12]:
d_x = namedtuple('d_x',['d_fi','d_qfi', 'd_R', 'd_p', ])
dd_x = namedtuple('dd_x',['dd_fi', 'dd_qfi', 'dd_R', 'dd_p',])
Y_x = namedtuple('Y_x', ['Yp', 'Yqfi', 'fi'])
h_x = namedtuple('h', ['hx', 'hy', 'hz'])
Hy_x = namedtuple('Hy_x', ['Hyp', 'Hyqfi', 'Hfi'])

In [15]:
Index = lambda epoch: next(key for key, val in Bgln[epoch].items() if val.el == np.max([val.el for key, val in Bgln[epoch].items()]))

In [49]:
sv_gln0 = [47, 50, 55, 57]
sv_gln_ = [5, 21, 18, 3]

JGL = len(sv_gln0)
#1300 - 2100
ts = 1300
te = 2100
Range = range(ts, te+1)

dX = {epoch: {sv: d_x(Rgln[epoch][sv].cp - Bgln[epoch][sv].cp,
                        (Rgln[epoch][sv].cp - Rgln[ts][sv].cp) - (Bgln[epoch][sv].cp - Bgln[ts][sv].cp),
                        Rgln[epoch][sv].R  - Bgln[epoch][sv].R,
                        Rgln[epoch][sv].pr*C - Bgln[epoch][sv].pr*C,) for sv in sv_gln0} for epoch in Range}

el = 57
ddX = {epoch: {sv: dd_x(dX[epoch][sv].d_fi - dX[epoch][el].d_fi,
                        (dX[epoch][sv].d_qfi - dX[epoch][el].d_qfi)*lam,
                          dX[epoch][sv].d_R / lam_gln[sv] - dX[epoch][el].d_R / lam_gln[el],
                          (1 / (len(sv_gln0) * (C / (F_gln[sv] - F_gln[el])))) * np.sum([dX[epoch][svm].d_R - dX[epoch][svm].d_p for svm in sv_gln0])) for sv in sv_gln0 if sv != el} for epoch in  Range}

ddM = {epoch: {sv: ddX[epoch][sv].dd_fi - ddX[epoch][sv].dd_R + ddX[epoch][sv].dd_p for sv in sv_gln0 if sv != el} for epoch in  Range}
ddMsv = {sv: [ddM[epoch][sv]-ddM[Range[0]][sv] for epoch in Range] for sv in sv_gln0 if sv != el}

In [50]:
(1 / (len(sv_gln0) * (C / (F_gln[sv] - F_gln[el])))) * np.sum([dX[epoch][svm].d_R - dX[epoch][svm].d_p for svm in sv_gln0])

NameError: name 'sv' is not defined

In [51]:
p = figure(width = 1000, plot_height = 400,  title="Неопределенное целое b1-"+filename[5:7]+' для ГЛОНАСС', 
           x_axis_label='Эпоха', y_axis_label='М',
           tools=TOOLS, toolbar_location="below",)

for usi, color in zip(sv_gln0, Category10[10]):
    if usi == el: continue
    p.line(Range, ddMsv[usi],
           color = color, alpha = 0.8, muted_color=color, muted_alpha=0.2, legend=str(usi), line_width = 1)
p.legend.click_policy="mute"
p.legend.orientation = "horizontal"
#p.xaxis.bounds = (1300, 2100)
show(p)

In [52]:
q

NameError: name 'q' is not defined

In [ ]:
# {epoch: {sv: for sv in sv_gps0} for epoch in Range}
# np.matrix([ (  ) for sv in sv_gps0 if sv != el])

In [55]:
Yp0 = {epoch: np.matrix([ ( dX[epoch][sv].d_R - dX[epoch][el].d_R - ddX[epoch][sv].dd_p ) for sv in sv_gln0 if sv != el]).T for epoch in Range}
Yqfi0={epoch: np.matrix([ ( dX[epoch][sv].d_R - dX[ts][sv].d_R - (dX[epoch][el].d_R - dX[ts][el].d_R) - ddX[epoch][sv].dd_qfi )  for sv in sv_gln0 if sv != el]).T for epoch in Range}
Yfi0 ={epoch: np.matrix([ (dX[epoch][sv].d_R / lam_gln[sv] - dX[epoch][el].d_R / lam_gln[el] - ddX[epoch][sv].dd_fi - (1 / (len(sv_gln0) * (C / (F_gln[sv] - F_gln[el])))) * np.sum([dX[epoch][svm].d_R - dX[epoch][svm].d_p for svm in sv_gln0]) ) for sv in sv_gln0 if sv != el]).T for epoch in Range}

In [56]:
# 15.12 Постваивл вместо Rgps - Bgps
dh = {epoch: {sv: h_x( (Bgln[epoch][sv].X - Bgln[epoch][sv].x) / Bgln[epoch][sv].R,
                      (Bgln[epoch][sv].Y - Bgln[epoch][sv].y) / Bgln[epoch][sv].R,
                      (Bgln[epoch][sv].Z - Bgln[epoch][sv].z) / Bgln[epoch][sv].R,) for sv in sv_gln0} for epoch in Range}

In [57]:
HYp0 = {epoch: np.matrix([(dh[epoch][sv].hx - dh[epoch][el].hx,
                           dh[epoch][sv].hy - dh[epoch][el].hy,
                           dh[epoch][sv].hz - dh[epoch][el].hz) for sv in sv_gln0 if sv != el]) for epoch in Range}

In [58]:
HYqfi0 = {epoch: np.matrix([*zip([dh[epoch][sv].hx - dh[ts][sv].hx - (dh[epoch][el].hx - dh[ts][el].hx) for sv in sv_gln0 if sv != el],
                                 [dh[epoch][sv].hy - dh[ts][sv].hy - (dh[epoch][el].hy - dh[ts][el].hy) for sv in sv_gln0 if sv != el],
                                 [dh[epoch][sv].hz - dh[ts][sv].hz - (dh[epoch][el].hz - dh[ts][el].hz) for sv in sv_gln0 if sv != el])]) for epoch in Range}

In [62]:
Hfi0 = {epoch: HYp0[epoch] / lam    for epoch in Range}

In [63]:
Y0 = {epoch: np.concatenate( (Yp0[epoch], Yqfi0[epoch]) , axis = 0) for epoch in Range}

In [64]:
H0 = {epoch: np.concatenate( ( (HYp0[epoch], HYqfi0[epoch], Hfi_GP0[epoch]) ) , axis = 0) for epoch in Range}

In [66]:
ddx, ddy, ddz = Rgln[te][sv_gln0[0]].x - Bgln[te][sv_gln0[0]].x, Rgln[te][sv_gln0[0]].y - Bgln[te][sv_gln0[0]].y, Rgln[te][sv_gln0[0]].z - Bgln[te][sv_gln0[0]].z

dT = np.matrix([ddx, ddy, ddz]).T

In [ ]:
us = 0
d = figure(width = 1200, plot_height = 250)
# d.line(Range, [np.array(np.abs( dY0[te].Yqfi - HYqfi0[te] * (dT) ))[0][0] for te in Range], color = 'red', legend = '6')
# d.line(Range, [np.array(np.abs( dY0[te].Yqfi - HYqfi0[te] * (dT) ))[1][0] for te in Range], color = 'blue', legend = '24')
# d.line(Range, [np.array(np.abs( dY0[te].Yqfi - HYqfi0[te] * (dT) ))[2][0] for te in Range], color = 'green', legend = '25')
# d.line(Range, [np.array(np.abs( dY0[te].Yqfi - HYqfi0[te] * (dT) ))[3][0] for te in Range], color = 'orange', legend = '30')
# d.line(Range, [np.array(dY0[te].fi )[us][0] - np.array(dY0[0].fi )[us][0] for te in Range], legend = 'fi_GP')
# d.line(Range, [np.array( Hfi_GP0[te] * (dT) )[us][0] - np.array( Hfi_GP0[0] * (dT) )[us][0] for te in Range], color = 'red', legend = f'Hfi * dTetta   sv:{sv_gps0[us]}')
d.line(Range, [np.array(np.abs( dY0[te].fi - Hfi_GP0[te] * (dT)))[us][0] -np.array(np.abs( dY0[0].fi - Hfi_GP0[0] * (dT)))[us][0] for te in Range], legend = f'разнциа sv:{sv_gps0[us]}')
d.legend.orientation = "horizontal"
d.legend.location = "top_left"
show(d)

In [68]:
print(np.abs(Y0[ts+60] - (H0[ts+60] * dT)[:2*(JGL-1)])[-(JGL-1):])

print(np.abs( dY0[ts+60].fi - Hfi0[ts+60] * (dT) ))

[[0.48457322]
 [0.34172093]
 [0.09221708]]
[[268.9929702 ]
 [172.61529213]
 [ 71.49217635]]


In [ ]:
q

In [ ]:
Rp = np.matrix(np.eye((JGP))* (1.15**2))
Rfi = np.matrix(np.eye((JGP))* (0.015**2))
Rqfi = 2 * Rfi

In [ ]:
R1p = 2 * Rp
R1fi = 2 * Rfi
R1qfi = 2 * Rqfi
RW1qfi = R1qfi * lam**2

In [ ]:
TR2 = np.eye((JGP-1))
TR2 = np.concatenate((TR2, np.matrix(np.ones((JGP-1))*-1).T), axis = 1)
#TR2[sv_gps0.index(el)][sv_gps0.index(el)] = 0
#TR2 = np.mqatrix(TR2)
#TR2 = np.delete(TR2, sv_gps0.index(el), 1)

In [ ]:
R2p = TR2 * R1p * TR2.T
R2fi = TR2 * R1fi * TR2.T
R2qfi = TR2 * RW1qfi * TR2.T
LAM = np.matrix(np.eye(JGP)*lam)
CRqfifi = 2 * TR2 * LAM * Rfi * TR2.T

In [ ]:
Null = np.matrix(np.zeros((JGP-1, JGP-1)))

In [ ]:
R_GP = np.concatenate((np.concatenate((R2p, Null, Null), axis = 0),
                np.concatenate((Null, R2qfi, CRqfifi.T), axis = 0),
                np.concatenate((Null, CRqfifi, R2fi), axis = 0)), axis = 1)

In [ ]:
B = R_GP.I

In [ ]:
D = B - B * H_GP0[te] * (H_GP0[te].T * B * H_GP0[te]).I * H_GP0[te].T * B

In [ ]:
#D = np.arange(12*12).reshape((12,12))

In [ ]:
Dpp = D[:(JGP-1)*2,:(JGP-1)*2]

Dqp = D[(JGP-1)*2:,:(JGP-1)*2]

Dpq = D[:(JGP-1)*2,(JGP-1)*2:]

Dqq = D[(JGP-1)*2:,(JGP-1)*2:]

In [ ]:
kzv = (-1) * Dqq.I * Dqp * Y_GP0[te] - dY0[te].fi
KVF =lambda k: (k - kzv).T * Dqq * (k - kzv)

In [ ]:
io.savemat('PYtoMAT.mat', {'Dqq': Dqq, 'kzv': kzv})

In [ ]:
K = np.matrix(io.loadmat('MATtoPY')['kmin'], dtype = 'int')

In [ ]:
i = 1
j = i+1
kmin = K[:,j:j+1]
ksled = K[:,i:i+1]

In [ ]:
dT = (H_GP0[te].T * B * H_GP0[te]).I * H_GP0[te].T * B * np.concatenate((Y_GP0[te], dY0[te].fi + kmin), axis = 0)
print(dT, '  ', popr(filename))

In [ ]:
KNTR = (KVF(ksled) / KVF(kmin)).tolist()[0][0]
KNTR

In [ ]:
KVF(ksled)

In [ ]:
KVF(kmin)

In [ ]:
def Q(sv, ts, te):
    ddqfi = {sv: lam*(dqfi(sv, te) - dqfi(el, te)) for sv in sv_gps0}
    ddfi = {sv: dfi(sv, te) - dfi(el, te) for sv in sv_gps0}
    ddp = {sv: dp(sv, te) - dp(el, te) for sv in sv_gps0}
    
    Yp_GP = np.matrix([dR(sv, te)  - ddp[sv] for sv in sv_gps0 if sv!= el]).T
    HYp = np.matrix([(hx(te, sv) - hx(te, el),
                   hy(te, sv) - hy(te, el),
                   hz(te, sv) - hz(te, el)) for sv in sv_gps0 if sv != el])
    Yqfi_GP = np.matrix([dqR(sv, te, ts) - dqR(el, te, ts) - ddqfi[sv] for sv in sv_gps0 if sv!= el]).T
    HYqfix = [ hx(te, sv) - hx(ts, sv) - hx(te, el) + hx(ts, el)  for sv in sv_gps0 if sv != el]
    HYqfiy = [ hy(te, sv) - hy(ts, sv) - hy(te, el) + hy(ts, el)  for sv in sv_gps0 if sv != el]
    HYqfiz = [ hz(te, sv) - hz(ts, sv) - hz(te, el) + hz(ts, el)  for sv in sv_gps0 if sv != el]

    HYqfi = np.matrix([*zip(HYqfix, HYqfiy, HYqfiz)])
    fi_GP =  np.matrix([dR(sv, te) / lam - ddfi[sv] for sv in sv_gps0 if sv!= el]).T
    Hfi_GP = HYp / lam
    
    Y_GP = np.zeros((2*len(Yp_GP),1))
    Y_GP[:len(Yp_GP)] = Yp_GP
    Y_GP[len(Yp_GP):2*len(Yp_GP)] = Yqfi_GP
    Y_GP = np.matrix(Y_GP)
    
    H_GP = np.zeros((3*len(HYp),3))
    H_GP[:len(HYp)] = HYp
    H_GP[len(HYp):2*len(HYp)] = HYqfi
    H_GP[2*len(HYp):3*len(HYp)] = Hfi_GP
    H_GP = np.matrix(H_GP)
    
    ddx, ddy, ddz = Rgps[te][sv].x - Bgps[te][sv].x, Rgps[te][sv].y - Bgps[te][sv].y, Rgps[te][sv].z - Bgps[te][sv].z
    dTetta = np.matrix([ddx, ddy, ddz]).T
    
    return np.abs(Y_GP - (H_GP * dTetta)[:2*len(Yp_GP)])#[-len(Yp_GP):]

In [ ]:
#Dqq
[1769.95389611312 -960.302974544202 68.842325950905;
-960.302974544202 1944.51339220187 -178.926983405045;
68.842325950905 -178.926983405045 16.7690061520173]

In [ ]:
#kzv
[0.623344115996358; 4.175518871976; 51.7000426397589;]

In [ ]:
#lDlt
[16.76900615201726 0.32989935223315 0.08472742086124;
0.32989935223315 35.34447066872609 -0.38709318743381;
0.08472742086124 -0.38709318743381 45.45709469463232;]

In [ ]:
# d = figure(width = 1000, plot_height = 350,)
# d.line(Range, [Q(6, 0, step)[4][0] for step in Range], color = 'red', legend = str(6))
# d.line(Range, [Q(6, 0, step)[5][0] for step in Range], color = 'green', legend = str(24))
# d.line(Range, [Q(6, 0, step)[6][0] for step in Range], color = 'orange', legend = str(25))
# d.line(Range, [Q(6, 0, step)[7][0] for step in Range], color = 'yellow', legend = str(30))
# show(d)

In [ ]:
# p = figure(width = 1000, plot_height = 400,  title="Неопределенное целое b1-"+filename[5:7]+' для ГЛОНАСС', 
#            x_axis_label='Эпоха', y_axis_label='М',
#            tools=TOOLS, toolbar_location="below",)

# for usi, color in zip(sv_gps0, Category10[10]):
#     #if usi == el: continue
#     p.line(Range, [Bgps[epoch][usi].el for epoch in Range],
#            color = color, alpha = 0.8, muted_color=color, muted_alpha=0.2, legend=str(usi), line_width = 1)
# show(p)

In [ ]:
# fin = 0
# xr = 0.#2846214.026350547
# yr = 0.#2201612.497401512
# zr = 0.#5248760.89142531
# dRr = 0.

# def R(xr,yr,zr):
#     return np.sqrt((xr-X)**2+(yr-Y)**2+(zr-Z)**2)

# def H(xr,yr,zr):
#     Rr = R(xr,yr,zr)
#     hx = (xr-X)/Rr
#     hy = (yr-Y)/Rr
#     hz = (zr-Z)/Rr
#     return np.matrix([[hx[i], hy[i], hz[i], 1.] for i in range(len(hx))])

# def Xi(dRr,Rr):
#     xi = (PDj+dTj)*ct.c-dRr-Rr
#     #return np.array(list(map(lambda val: val**2, xi)))
#     return xi

# def dTett0(H,Xi):
#     xi = np.matrix(Xi).T
#     A = H.T*H
#     B = H.T
#     #return np.linalg.lstsq(H,xi,rcond=None)[0]
#     return np.linalg.lstsq(A,B*xi,rcond=None)[0]
#     #return np.linalg.lstsq(H,xi)[0]

# def step_ord(xr,yr,zr,dRr):
#     Rr = R(xr,yr,zr)
#     H0 = H(xr,yr,zr)
#     xi = Xi(dRr,Rr)
#     dT = dTett0(H0,xi)
#     xr += dT[0,0]
#     yr += dT[1,0]
#     zr += dT[2,0]
#     dRr += dT[3,0]
#     return xr,yr,zr,dRr,dT

# def coord_modif(xr,yr,zr):
#     Rr = R(xr,yr,zr)
#     tau8 = Rr/(ct.c)
#     alpha = omega3*tau8
#     xm = X + alpha*Y
#     ym = Y - alpha*X
#     zm = Z
#     return xm,ym,zm

# def PR(dTett0):
#     dT0 = np.abs(dTett0)
#     return dT0[0] < 1. and dT0[1] < 1. and dT0[2] < 1. and dT0[3] < 1.

# fR = lambda xr,yr,zr: np.sqrt((xr)**2+(yr)**2+(zr)**2)

# j = 0
# while fin != 1 and j<2000:
#     #print(R(xr,yr,zr))
#     xr,yr,zr,dRr,dT = step_ord(xr,yr,zr,dRr)
#     Rr = fR(*[xr,yr,zr])
#     #print("step {}".format(j+1))
#     #print('xr: ',xr,'\nyr: ',yr,'\t\tR: ',Rr,'\nzr: ',zr,'\ndRr: ',dRr,'\n')
#     j+=1
#     if PR(dT):
#         X,Y,Z = coord_modif(xr,yr,zr)
#         xr,yr,zr,dRr,dT = step_ord(xr,yr,zr,dRr)
#         print('uspeh')
#         print('xr: ',xr,'\nyr: ',yr,'\t\tR: ',Rr,'\nzr: ',zr,'\ndRr: ',dRr,'\n')
#         fin = 1

In [ ]:
# %%time
# Bgln = StatusGLN(B1, bgln)
# Rgln = StatusGLN(R1, rgln, dx, dy, dz)

# B = [*Bgln[0].keys()]
# B = [55, 57]
# B0 = [18, 3]
# B = sv_arr0
# B0 = [5, 21, 18, 3]

# Range = range(len(Rgln))
# #Range = range(1300,2100)
# dX0 = {epoch: {sv: d_x0(Rgln[epoch][sv].cp - Bgln[epoch][sv].cp,
#                         Rgln[epoch][sv].R  - Bgln[epoch][sv].R,
#                         Rgln[epoch][sv].pr*C - Bgln[epoch][sv].pr*C) for sv in B} for epoch in Range}

# el = 55
# ddX0 = {epoch: {sv: dd_x0(dX0[epoch][sv].d_fi - dX0[epoch][el].d_fi,
#                           dX0[epoch][sv].d_R / lam_gln[sv] - dX0[epoch][el].d_R / lam_gln[el],
#                           (1 / (len(B) * (C / (F_gln[sv] - F_gln[el])))) * np.sum([dX0[epoch][svm].d_R - dX0[epoch][svm].d_p for svm in B])) for sv in B if sv != el} for epoch in  Range}

# ddM0 = {epoch: {sv: ddX0[epoch][sv].dd_fi - ddX0[epoch][sv].dd_R + ddX0[epoch][sv].dd_p for sv in B if sv != el} for epoch in  Range}
# ddMsv0 = {sv: [ddM0[epoch][sv]-ddM0[Range[0]][sv] for epoch in Range] for sv in B if sv != el}